In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os 
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# Normalise images to fall into the 0-1 range
#X = (X/255)

In [5]:
model = tf.keras.applications.MobileNetV2()

base_input = model.input
base_output = model.layers[-2].output  # Get output before the classification layer

#да се оправи

final_output = layers.Dense(64)(base_output)  
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation="softmax")(final_output)

modified_model = keras.Model(inputs = base_input, outputs = final_output)
modified_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
img_size = 224
dataDirectory = "train/"
batch_size = 16

datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = datagen.flow_from_directory(
    dataDirectory,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=True
)

modified_model.fit(train_generator, epochs = 25)
modified_model.save('Final_model_95p07.keras')

Found 28709 images belonging to 7 classes.
Epoch 1/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 2056s 1s/step - accuracy: 0.7876 - loss: 0.5784
Epoch 2/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 2007s 1s/step - accuracy: 0.8084 - loss: 0.5286
Epoch 3/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 1994s 1s/step - accuracy: 0.8298 - loss: 0.4799
Epoch 4/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 1989s 1s/step - accuracy: 0.8450 - loss: 0.4364
Epoch 5/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 1981s 1s/step - accuracy: 0.8578 - loss: 0.3967
Epoch 6/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 1956s 1s/step - accuracy: 0.8719 - loss: 0.3643
Epoch 7/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 1909s 1s/step - accuracy: 0.8834 - loss: 0.3263
Epoch 8/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 1913s 1s/step - accuracy: 0.8947 - loss: 0.2998
Epoch 9/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 1904s 1s/step - accuracy: 0.9024 - loss: 0.2745
Epoch 10/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 1905s 1s/step - accuracy: 0.9065 - loss: 0.2589
Epoch 11/25
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 1914s 1s/step

In [17]:
modified_model = tf.keras.models.load_model('Final_model_95p07.keras')

In [11]:
import cv2
import numpy as np
import tensorflow as tf
import os

# за преразглеждане на тест папката след трениране на модела

new_model = tf.keras.models.load_model('Final_model_95p07.keras')

test_images_path = 'test/'
output_images_path = 'test/results'

if not os.path.exists(output_images_path):
    os.makedirs(output_images_path)

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

for emotion_folder in os.listdir(test_images_path):
    emotion_folder_path = os.path.join(test_images_path, emotion_folder)
    
    if os.path.isdir(emotion_folder_path):
        # Create corresponding output subfolder
        output_emotion_folder_path = os.path.join(output_images_path, emotion_folder)
        if not os.path.exists(output_emotion_folder_path):
            os.makedirs(output_emotion_folder_path)

        for filename in os.listdir(emotion_folder_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):  
                img_path = os.path.join(emotion_folder_path, filename)
                frame = cv2.imread(img_path)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = faceCascade.detectMultiScale(gray, 1.1, 4)

                for (x, y, w, h) in faces:
                    roi_gray = gray[y:y+h, x:x+w]
                    roi_color = frame[y:y+h, x:x+w]
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

                    final_image = cv2.resize(roi_color, (224, 224))
                    final_image = np.expand_dims(final_image, axis=0)
                    final_image = final_image / 255.0

                    Predictions = new_model.predict(final_image)
                    status = np.argmax(Predictions)

                    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
                    label = emotions[status]

                    cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

                # Save the image with predictions
                output_img_path = os.path.join(output_emotion_folder_path, filename)
                cv2.imwrite(output_img_path, frame)

cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━

In [15]:
# тук надолу всичко е наред с кода и тестването чрез камера

new_model = tf.keras.models.load_model('Final_model_95p07.keras')

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        final_image = cv2.resize(roi_color, (224, 224))
        final_image = np.expand_dims(final_image, axis=0)
        final_image = final_image / 255.0

        Predictions = new_model.predict(final_image)
        status = np.argmax(Predictions)

        emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        label = emotions[status]

        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        
    cv2.imshow("Face Emotion Recognition", frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━

KeyboardInterrupt: 